<a href="https://colab.research.google.com/github/WetSuiteLeiden/example-notebooks/blob/main/specific-experiments/review-algoritmeregister/algoritmeregister.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#if running in colab and you want to reproduce, run the following two, otherwise ignore this
!pip3 install wetsuite -U

In [ ]:
!python3 -m spacy download nl_core_news_lg

## Purpose of this notebook

For context, [algoritmes.overheid.nl](https://algoritmes.overheid.nl) is government organisations  about the algorithms they use in their day-to-day,
from automation to machine learning, in a bid to be more transparent towards citizens.

What if, based on just the descriptions,
we want to see whether things end up profiling individuals? 
We expect this primarily in applications related to social security, though there may be more.

And yes, [yhe algoritmes.overheid.nl site](https://algoritmes.overheid.nl) has topic filtering, 
and a decent search, but it's hard to know how complete the results are at any time

And while it does have text per case, based on a [publication standard asking some specific questions](https://algoritmes.pleio.nl/wiki/view/35c7eeaa-deee-47d3-b2cc-88f7c90b475c/invullijst-algoritmeregister-publicatiestandaard-100), it's in a few different places, and the way the answers are given does vary.

Even though there are 'only' five hundred items to check,
reviewing by hand would be slow busywork - we would be two thousand clicks further before we reviewed everything.

### Considering that task

We described three things to address:
- Turning five hundred _things_ on a site into something we can consume more easily 
- Ensure that we are considering all cases we care about
- Classify as worrisome or not

<!-- -->

Consuming the site as data is not too hard - the site also has CSV and XSLX export, both machine-readable enough, so we can start handling that as text data.
Sadly you cannot export the current filtered selection, so we will have to deal with all of it.

<!-- -->

In theory you can see the other two point as classification tasks.
- social topic or not? (as a filter)
- has worrisome phrasing or not? (probably to score)

While classification is automatic,
and large enough documents have a tendency to supply good-enough indicators,
in this case there is little text to work on.

This also makes phrasing important - just hedging your language well enough would probaly make something pass.
We suspect that even _if_ there were no issue in the time cost in reading all that,
people doing that busywork would probably still have trouble, and might even disagree.

Sure, clearer cases may have some red-flag words --  _but_ some cases are just too vague even forwell-informed human estimation. 
When the information isn't there for us even with a little more real-world knowledge, it's probably less there for machines.

In particular the last case of the next list suggests a real difference between 
"lack of topical words that make us worry; probably fine", and "lack of topical words; should ask for them".

---
Consider some real cases that we would consider investigating further:

* [Model van Bijstand naar Werk](https://algoritmes.overheid.nl/nl/algoritme/model-van-bijstand-naar-werk-gemeente-den-haag/71627856#verantwoordGebruik)
  - worrisome: 
    - the topic of 'bijstand'; terms like 'op maat', 'dossiers', 'decision tree', 'bepaalde segmenten'
    - the goal is to invite _specific_ people
  - alleviates?
    - 'eenmalig' suggests removal after suggestion
    - system _advises_ an invitation, does not decide 

* [Heronderzoeken Uitkeringsgerechtigden](https://algoritmes.overheid.nl/nl/algoritme/heronderzoeken-uitkeringsgerechtigden-gemeente-rotterdam/36585638#verantwoordGebruik)
  - worrisome: the topic of 'uitkering'; terms like 'kansberekening', 'risico-inschattingsgetal', 'historische gegevens', 'voorspellend', 'gezinssituatie'
  - alleviates: system _advises_ a review, does not decide or execte one?

* [Onderzoekswaardigheid: Slimme check levensonderhoud](https://algoritmes.overheid.nl/nl/algoritme/onderzoekswaardigheid-slimme-check-levensonderhoud-gemeente-amsterdam/95794697#verantwoordGebruik)
  - worrisome: terms like 'score', 'onderzoek', 'leeftijd', 'geboorteland'
  - alleviates:
    - system _advises_ a review, does not make decisions  ('onderzoekswaardig')
    - (explains that it addresses ethics in that it should be more equal-opportunity about scrutiny, though not exactly how much profiling is or isn't involved in that)

* [Werkverkenner](https://algoritmes.overheid.nl/nl/algoritme/werkverkenner-uitvoeringsinstituut-werknemersverzekeringen/11248112#verantwoordGebruik)
  - worrisome:
    - topic of 'uitkering', terms like 'score', the [15-item list of personal information that it uses](https://algoritmes.overheid.nl/nl/algoritme/werkverkenner-uitvoeringsinstituut-werknemersverzekeringen/11248112#werking) 
  - alleviates:
    - said to involve checking the information that led to the invitation
    - external verification of ethics

---
Possibly okay?

* [Wmo-voorspelmodel](https://algoritmes.overheid.nl/nl/algoritme/wmovoorspelmodel-gemeente-den-haag/97246956#verantwoordGebruik)
  - worrisome: 'voorspelmodel' 
  - alleviates: results are about collective, not personal use; basis might still be personal info but 
  'open data' suggests not

* [Vroegsignalering](https://algoritmes.overheid.nl/nl/algoritme/vroegsignalering-gemeente-roosendaal/73933449#verantwoordGebruik)
  - worrisome: 'schuldeisers', 'automatisch', 'persoonsgegevens'
  - alleviates: 'advies' suggests suggestion rather that decision system; 'handmatig' suggests this is about automating, not about doing things particularly differently

* [Rechten Rotterdammers](https://algoritmes.overheid.nl/nl/algoritme/rechten-rotterdammers-gemeente-rotterdam/33569518#verantwoordGebruik)
  - worrisome: topic like uitkering, 'beslisregels', 'toekennen' based on its output
  - alleviates: 'advies', 'medewerker'+'ondersteunt'
Regelgebaseerd algoritme stelt rechten vast (geen risicoscores) 

* [Automatische kwijtschelding](https://algoritmes.overheid.nl/nl/algoritme/automatische-kwijtschelding-gemeente-rotterdam/54699221#verantwoordGebruik)
  - worrisome: 'combineren'+'informatie', 'inkomstengegevens', 'financiële' en 'huishouden' 
  - alleviates: 'opt-in'; 'handmatig'

---
Unsure?

* [DIAfragma](https://algoritmes.overheid.nl/nl/algoritme/diafragma-gemeente-amsterdam/38895425#verantwoordGebruik)
  - worrisome: terms like 'integraal klantbeeld', 'BSN-nummers', 'geslacht', 'koppeling',  [list of 10+ pieces of personal information](https://algoritmes.overheid.nl/nl/algoritme/diafragma-gemeente-amsterdam/38895425#werking) 
  - alleviates: specifically avoids using some information that might lead to discrimination?

* [Sociaal Domein: PKO Kennissystemen (Proces & Kennisondersteuning)](https://algoritmes.overheid.nl/nl/algoritme/sociaal-domein-pko-kennissystemen-proces-kennisondersteuning-gemeente-arnhem/65732191#verantwoordGebruik)
  - worrisome: topic of uitkering, terms like 'beslisboom', 'controleert', 'door het systeem getrokken conclusies' 
  - alleviates: 'nalopen'
  - not a lot of contentful words to go on, though?




---

What if we limit ourself to marking how interesting cases are for us to potentially _look at_,
rather that decide _how bad they are_?

We only support a human in making their decisions easier
(the same thing we want to see in some of these algoritms themselves),

To scrape as much as we can from minimal text, 
we might look at worrisome words, which verbs appear in the same sentence? Paragraph?

There sould probably be some suggestion of phrases to include,
based on being related to what you have already decided is good or bad,
or just on being contentful words not currently under consideration.

---
That said, this approach also comes with different fundamental limitations. 

For example, even smarter NLP might miss negations (even recent developments, LLMs, used to be infamously bad at seeing negations).

Consider detecting 'geautomatiseerde besluitvorming' and miss that the words before were 'Er is geen sprake van';
- You might see 'BSN-nummers' and 'koppeling' but miss the words 'zonder verdere' inbetween.
- You might see 'handmatig' but not tell whether it is the thing we are not doing anymore, or ensuring is still there.

Things like "Risico's bij het gebruik van het algoritme zijn mede vanwege de mogelijkheid om in te grijpen op de door het systeem getrokken conclusies niet noemenswaardig, waarmee de proportionaliteit in orde is." are a feat of sentence nesting, and we would like an extra connecting argument that actually leads to that final "it's probably fine".


But then, if we're not even looking for interpretation like negatives, we are _certain_ to miss those differences.
That said, it is not necessarily a bad thing to bring up these cases, if only because some cases lie fully in context.

"Het algoritme kan alleen automatisch een aanvraag/aangifte goedkeuren. Een aanvraag/aangifte afkeuren kan alleen de ambtenaar." may sound like good bias, or bad,
depending on what it is we're okaying.

## The data

### Fetching and parsing data

In [1]:
import csv, io, random, re, math
import wetsuite.helpers.net
import wetsuite.helpers.strings
import wetsuite.helpers.spacy
import wetsuite.helpers.lazy

In [2]:
## Fetch that data as CSV
csv_bytes = wetsuite.helpers.net.download( 'https://algoritmes.overheid.nl/api/downloads/NLD?filetype=csv', timeout=60 )   # higher timeout because it may take take 10 to 20 seconds to produce that
# (you can ignore this) 
# (do some manual decoding because it seems to has UTF-8 with a BOM which is not invalid but is sort of pointless, and not something a lot of things look for / look to remove)
csv_str = csv_bytes.decode('utf-8').lstrip('\ufeff') # Remove that, move on.


## Parse that CSV
# Now we have CSV in a (unicode) string.
# CSV is not a single standard, it has flavours that we need to get the right one of.
# This seems good for this data:
class AlgoritmRegisterCSVDialect(csv.Dialect):
    header         = True
    lineterminator = '\n'
    delimiter      = ','
    doublequote    = True
    quotechar      = '"'
    quoting        = csv.QUOTE_ALL

csv_parsed = csv.DictReader( io.StringIO(csv_str) , dialect=AlgoritmRegisterCSVDialect())
algodict_list = list( csv_parsed ) 

In [3]:
# we now have a list of dicts. Just to point that out, show the first:   (note that all items will share the same keys, due to where it came from)
algodict_list[0] 

{'name': 'Public Eye',
 'organization': 'Gemeente Amsterdam',
 'description_short': 'Amsterdam is een drukke stad. Dit kan soms leiden tot verkeersonveilige situaties. Door data te verzamelen over de aantallen voetgangers is het mogelijk om maatregelen te treffen, waardoor de drukte in goede banen geleid kan worden.',
 'category': 'Organisatie en bedrijfsvoering, Economie, Ruimte en infrastructuur',
 'website': 'https://algoritmeregister.amsterdam.nl/public-eye/',
 'status': 'Buiten gebruik',
 'goal': 'Amsterdam is een drukke stad. Dit kan soms leiden tot verkeersonveilige situaties. Door data te verzamelen over de aantallen voetgangers is het mogelijk om maatregelen te treffen, waardoor de drukte in goede banen geleid kan worden. Hierdoor blijft de stad comfortabel, bereikbaar en verkeersveilig. Als een situatie door te grote drukte onveilig wordt, kan de gemeente ingrijpen. Dit gebeurt bijvoorbeeld door digitale informatieborden te plaatsen, zodat mensen weten welke routes ze moeten 

### Inspecting data

In [3]:
print( 'Number of algorithm:', len(algodict_list) )

Number of algorithm: 536


In [8]:
# ...so showing _all_ data like above would be messy. Let's try something more compact, like a table.
# pandas is a handy tool to work with table-like data
#   and deals with lists of dicts (as we just read in) by making a column for each key -- so in this case happens to need no further instruction

import pandas 
# just for this case, show just a few rows, but all columns (we're pointing out how sparse this is)
with pandas.option_context('display.min_rows',5,  'display.max_columns',None): 
    df = pandas.DataFrame(algodict_list) 
    display(df)

,name,organization,description_short,category,website,status,goal,proportionality,lawful_basis,standard_version,url,contact_email,lang,publiccode,source_data,methods_and_models,human_intervention,risks,provider,process_index_url,tags,source_id,begin_date,end_date,impacttoetsen,publication_category,lawful_basis_grouping,impacttoetsen_grouping,source_data_grouping,algorithm_id,type,iama_description,uuid,lawful_basis_link,source_data_link,department,impact,decision_making_process,documentation,competent_authority,iama,dpia,dpia_description,objection_procedure,area,revision_date,description,application_url,mprd,monitoring,performance_standard
0,Public Eye,Gemeente Amsterdam,Amsterdam is een drukke stad. Dit kan soms lei...,"Organisatie en bedrijfsvoering, Economie, Ruim...",https://algoritmeregister.amsterdam.nl/public-...,Buiten gebruik,Amsterdam is een drukke stad. Dit kan soms lei...,,,1.0,,Algoritmen@amsterdam.nl,nld,,Trainingsdata: Marineterrein Met trainingsdata...,Architectuur van het model Een camera maakt vi...,Aan de hand van de trainingsdata wordt de kwal...,De videobeelden die gebruikt worden door Publi...,,,Crowd monitoringCrowd countingopen-source crow...,,,,,Impactvolle algoritmes,,,,38748497,,,,,,,,,,,,,,,,,,,,,
1,Dynamisch Kabelmodel,Liander N.V.,Het dynamisch kabelmodel is een model waarmee ...,Ruimte en infrastructuur,,In gebruik,Anno 2024 is er veel congestie op het elektri...,Gebruik van dit product ondersteunt de organis...,,1.0,,algoritmes@alliander.com,nld,,Het dynamisch kabelmodel maakt gebruik van div...,De inputgegevens die genoemd zijn in de Gegeve...,De mens maakt altijd de keuze over wat er word...,Risicobeheer voor het DKM-algoritme bestaat ui...,,,,,2022-01,,,Impactvolle algoritmes,,,"1: LHM, https://nhi.nu/modellen/lhm/#:~:text=H...",12678378,,,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,Telefonisch Innen (Wet Wahv),Centraal Justitieel Incassobureau,Eén van de belangrijkste maatschappelijke opga...,Wahv,https://www.cjib.nl/telefonisch-innen,In gebruik,,,,0.1,,,,,,,,,,,,,,,,,,,,41914566,Regelgebaseerd,,,,,nvt,,,,,,,,,,,,,,,
535,Vergunningverlening; Wonen,Gemeente Utrecht,Het uitvoeren van de algemene leefbaarheidstoe...,,,,,,,0.1,,,,,,,Het algoritme geeft informatie en dit stelt ee...,,,,,,,,,,,,,18113668,,,,,,"Vergunningen, Toezicht en Handhaving (VTH)",,Het algoritme combineert en presentreert vastg...,,,,,,,,,,,,,


In [9]:
# inspect how common some values are in each field
display( df.value_counts('status') )

status
In gebruik         458
In ontwikkeling     44
Buiten gebruik      33
                     1
Name: count, dtype: int64

In [11]:
display( df.value_counts('publication_category') )

publication_category
Impactvolle algoritmes    249
Overige algoritmes        248
Hoog-risico AI-systeem     23
                           16
Name: count, dtype: int64

In [12]:
catcounts = df.value_counts('category')
display( catcounts[catcounts>1] )

category
Organisatie en bedrijfsvoering                                             179
Sociale zekerheid                                                           60
                                                                            41
Openbare orde en veiligheid                                                 39
Verkeer                                                                     26
Ruimte en infrastructuur                                                    25
Overheidsfinanciën                                                          16
Natuur en milieu                                                            13
Wonen                                                                        9
Economie, Ruimte en infrastructuur                                           8
Zorg en gezondheid                                                           8
Economie, Ruimte en infrastructuur, Verkeer                                  6
Economie                                   

## Working our way to do that task

### Selecting just the interesting text fields

In [13]:
# This is the list we have
print( 'Columns/key names:', list( algodict_list[0].keys() ) )

Columns/key names: ['name', 'organization', 'description_short', 'category', 'website', 'status', 'goal', 'proportionality', 'lawful_basis', 'standard_version', 'url', 'contact_email', 'lang', 'publiccode', 'source_data', 'methods_and_models', 'human_intervention', 'risks', 'provider', 'process_index_url', 'tags', 'source_id', 'begin_date', 'end_date', 'impacttoetsen', 'publication_category', 'lawful_basis_grouping', 'impacttoetsen_grouping', 'source_data_grouping', 'algorithm_id', 'type', 'iama_description', 'uuid', 'lawful_basis_link', 'source_data_link', 'department', 'impact', 'decision_making_process', 'documentation', 'competent_authority', 'iama', 'dpia', 'dpia_description', 'objection_procedure', 'area', 'revision_date', 'description', 'application_url', 'mprd', 'monitoring', 'performance_standard']


In [16]:
# Looking at that short table above, most keys do note seem interesting.
#  starting from the list of keys above, remove things we do not find interesting.
#  the commented-out things are still _potentially_ useful - play with it to see
our_key_choices = [
        'category',
        'algorithm_id',           # keep this around, it lets us later link to the site again
        #'publication_category',
        'name', 
        'organization',
        'description_short',
        'goal',
        'proportionality',
        'source_data',
        'methods_and_models',
        'human_intervention',
        'risks',
        'tags',
        #'status',
        #'begin_date', 'end_date',
        #'impacttoetsen',
        #'impacttoetsen_grouping',
        #'source_data_grouping', 
]

# do the selection as a pandas table, so we can get a quick overview of what we made
smaller_table = df[our_key_choices]
display(smaller_table)

,category,algorithm_id,name,organization,description_short,goal,proportionality,source_data,methods_and_models,human_intervention,risks,tags
0,"Organisatie en bedrijfsvoering, Economie, Ruim...",38748497,Public Eye,Gemeente Amsterdam,Amsterdam is een drukke stad. Dit kan soms lei...,Amsterdam is een drukke stad. Dit kan soms lei...,,Trainingsdata: Marineterrein Met trainingsdata...,Architectuur van het model Een camera maakt vi...,Aan de hand van de trainingsdata wordt de kwal...,De videobeelden die gebruikt worden door Publi...,Crowd monitoringCrowd countingopen-source crow...
1,Ruimte en infrastructuur,12678378,Dynamisch Kabelmodel,Liander N.V.,Het dynamisch kabelmodel is een model waarmee ...,Anno 2024 is er veel congestie op het elektri...,Gebruik van dit product ondersteunt de organis...,Het dynamisch kabelmodel maakt gebruik van div...,De inputgegevens die genoemd zijn in de Gegeve...,De mens maakt altijd de keuze over wat er word...,Risicobeheer voor het DKM-algoritme bestaat ui...,
2,"Recht, Zorg en gezondheid, Natuur en milieu",47653623,Persoonlijke Impact Analyse - Immateriële Schade,Instituut Mijnbouwschade Groningen,Bij het IMG bestaat de regeling Immateriële Sc...,Doel: de PIA wordt door het IMG gebruikt om te...,Voordelen:De PIA maakt het mogelijk om individ...,Gegevens die door een aanvrager moeten worden ...,De aanvrager krijgt tijdens een aanvraag voor ...,De uitkomst van de PIA heeft invloed op de hoo...,Bij de ontwikkeling van de PIA is een DPIA uit...,Immateriële SchadevergoedingIMSPIA
3,"Ruimte en infrastructuur, Economie",16268148,Druktebeeld (NL),Gemeente Amsterdam,"Druktebeeld is een webapp, die een beeld geeft...",,,CMSA Het Crowd Monitoring Systeem Amsterdam (C...,Modelarchitectuur Voor de visualisaties in Dru...,Er worden geen geautomatiseerde besluiten geno...,De geïdentificeerde risico’s zijn:\r\n\r\nBesl...,
4,Organisatie en bedrijfsvoering,76218516,Permanente Monitor Dubbelinschrijvingen (PMD) ...,Rijksdienst voor Identiteitsgegevens,De Permanente Monitor Dubbelinschrijvingen is ...,Het doel van de algoritmen is dubbele inschrij...,We controleren dubbelinschrijvingen in de BRP ...,De identificerende gegevens van een persoon: v...,Technische documentatie is opvraagbaar bij RvI...,De PMD-algoritmen gebruiken we om op een repro...,Het onterecht afvoeren van een BRP-inschrijving.,"Rijksdienst voor Identiteitsgegevens, RvIG, BRP"
...,...,...,...,...,...,...,...,...,...,...,...,...
531,,23448142,Sancties,Sociale Verzekeringsbank,Een beslistool helpt onze medewerkers om te be...,Algoritmes helpen bij moeilijke beslissingen w...,,Onze belangrijkste bronnen zijn uw gegevens in...,Beslisboom (rule-based),,,
532,,96585577,Vrijwillige verzekering AOW en Anw,Sociale Verzekeringsbank,Een algoritme berekent de hoogte van de premie...,Algoritmes helpen bij het nuttig uitvoeren van...,,Onze belangrijkste bronnen zijn:\r\n\r\npersoo...,Beslisboom (rule-based),,,
533,,28915384,Preventie & Handhaving,Sociale Verzekeringsbank,Met dit model voorspellen wij hoe groot de kan...,Door het SWAN-model kunnen wij ons richten op ...,,Onze belangrijkste bronnen zijn interne gegeve...,Zelflerend,,Wij toetsen onze modellen altijd of ze voldoen...,
534,Wahv,41914566,Telefonisch Innen (Wet Wahv),Centraal Justitieel Incassobureau,Eén van de belangrijkste maatschappelijke opga...,,,,,,,


### Making a new structure with that data, and some new fields

In [17]:
# Say we want to flatten/merge all that text to one string
# The column selection part is easy to do in pandas (we just did, above),
#   but addressing rows in pandas is awkward,
#   so we go back to the earlier data in python types, lis.  Same key names, after all.

ourdata = {}  # { algo_id : {  key:value, key:value, ... } }
for item_dict in algodict_list:
    algo_id = item_dict.get('algorithm_id')

    # We make one new field, which contains all interesting text, to work on later. This code pastes them together.
    text_fragments = [] 
    for text_field_name in our_key_choices: 
        if text_field_name not in ('algorithm_id'):
            text_fragments.append( item_dict.get(text_field_name) )

    ourdata[algo_id] = { # we may not use all these fields, but it's handy
        'algo_id':algo_id, 
        'titel':item_dict.get('name'),
        'category':item_dict.get('category'),
        'textlist':text_fragments, 
        # preparing for what we're doing later:
        'sociaal':set(), 
        'worrying':set(),
        'alleviates':set(),
    }

In [37]:
# show a random one of them, to show what we just made
display( random.choice( list(ourdata.items()) ) )

('59432597',
 {'algo_id': '59432597',
  'titel': "Scanauto's parkeerhandhaving",
  'category': 'Verkeer',
  'textlist': ['Verkeer',
   "Scanauto's parkeerhandhaving",
   'Gemeente Utrecht',
   "Door de inzet van scanauto's zorgen we voor een betere en efficiëntere manier van parkeerhandhaving. Daardoor kan je als bewoner eerder een parkeerboete krijgen als je in overtreding zou zijn, dan voorheen het geval was.",
   "Het controleren met scanauto's of er sprake is van foutparkeren",
   "Resultaten Uthiek Assessment: door de inzet van scanauto's zorgen we voor een betere en efficiëntere manier van parkeerhandhaving. Hiermee voeren we als gemeente onze wettelijke taak op de juiste manier uit. Dit draagt bij aan de waarde Machtsverhoudingen. De waarde Privacy is NIET in het geding, omdat persoonsgegevens van bewoners (beelden van kenteken auto's) niet worden opgeslagen en direct in de camera van de scanauto worden verwijderd. Vanuit de waarde Controle op technologie was dit een vereiste vo

### Aside (you can skip this): Noting the 'enthusiastic dutch compounding makes for out-of-vocabulary words' problem

NLP likes to see words as 'character sequences without spaces' because that's a lot easier to tokenize into (and then assign a unique number for each token).

Dutch likes smashing together words any chance it gets, which means it's better than other languages
at making words that aren't very common in training data, known at all, and/or are hard to estimate meaning of.

We normally might not care so much, because for larger documents we are likely to catch the topic 
even if we miss a few words, but here we might need every word we can get. 

A thing to keep in mind. We may use libaries like spacy sparingly/informedly.
<!--
#     # for nc in doc.noun_chunks:
#     #     ph.add( wetsuite.helpers.strings.remove_deheteen( nc.text ) )
#     # for ent in doc.ents:
#     #     ph.add( wetsuite.helpers.strings.remove_deheteen( ent.text ) )
#     # if len(ph)>0:
#     #     data[id]['ph'].extend( ph )

#     # data[id]['ph'] = ( wetsuite.helpers.strings.ordered_unique(data[id]['ph'], case_sensitive=False) )

# pprint.pprint( data )
# -->

In [50]:
# get a random example case
random_case_dict = random.choice( list(ourdata.values()) )
random_case_text = ' '.join( random_case_dict['textlist'] )

doc = wetsuite.helpers.lazy.spacy_parse( random_case_text )
display( wetsuite.helpers.spacy.notebook_content_visualisation(doc) )  # that function already happens to highlight out-of-vocabulary words

### Filtering text for sociale zekerheid; identifying words of interest

Assuming we don't trust the field/filter the site gives us, 
or at least want to check it leaves out nothing, let's try our own, and compare.

Right now these are used as substrings, so will match even if part of larger words (which is why inflections are removed from this list, or should be),
done intentionally to alleviate the compounding issue - e.g. bijstand will match bijstandnorm, bijstandgerechtigd, etc.  
There's some extra code to then extract whatever the whole word was we matched a part from.

In [18]:
# substrings that indicate this is related to social security
social_indicators = '''bijstand
uitkering
werkloosheidswet
IOAZ
Wet inkomensvoorziening oudere en gedeeltelijk arbeidsongeschikte gewezen zelfstandigen 
IOAW
Wet inkomensvoorziening oudere en gedeeltelijk arbeidsongeschikte werkloze werknemers
IOW
Wet inkomensvoorziening oudere werklozen
de Wmo
Wet maatschappelijke ondersteuning
WWB
wet werk en bijstand
Wet LB
Wet op de loonbelasting
Awir
Algemene wet inkomensafhankelijke regelingen
Ziektewet
Participatiewet
UWV
AOW
arbeidsongeschik
arbeidsbeperk
arbeidsvermogen
WAJONG
schuldhulpverlen
loonwaarde
levensonderhoud
re-integratie
reïntegratie
werkhervat
Hulp bij het Huishouden
Ondersteuning thuis
betalingsachterstand
Kinderbijslag
kinderopvang
toeslagpartner
Werk en Inkomen
ziekmeld
Rechtsbijst
kwijtschel
zorgaanbied
ziekengeld
woonlandbeginsel
alimentatie
Sociale Verzekerings
Sociale zekerheid'''.lower().split('\n') # ww   sociale


# substrings that make us pay attention.  Again: this is NOT very robust
worrisome_indicators = '''voorspel
beslis
burgerservicenummer
BSN
Sofinummer
Wet basisregistratie personen
BRP
profiler
individu
risico
score
gestuurd
onderzoek
kansberekening
inschattingsgetal
dossier
uw gegevens
combineren
woonadres
inkomst(en)?gegeven
vermogensgegeven
voertuiggegeven
financi[eë]le gegeven
financi[ëe]le informatie
gezinss?ituatie
leeftijd
functie
huishouden
geboorte
persoonsgegeven
naamgegeven
geboortegegeven 
ziekmelding
leeftijd
geboorteland
nationaliteit
decision tree
beslisbo
rule-based
op maat
bijstand
uitkeringsgerechtig
werkzoeken
zonder tussenkomst van
historische gegeven
schuldeiser
schuldenaa?r
toeslagen
heronderzoek
datamodel'''.lower().split('\n') # heronderzoek


# substrings that suggest people considered (not) profiling
alleviating_indicators = '''handmatig
handmatig bekijk
handmatige? controle
niet leid
advie?[sz]
eenmalig
ondersteun
foutgevoel
Algemene Verordening Gegevensbescherm
AVG
profilering
discrimin
kwijtscheld
niet zelfler
privacy
privacy( impact)? assessment'''.lower().split('\n')

### Match those terms in that data

In [19]:
# we could work on only the things already tagged 'sociale zekerheid' to make the output shorter
#filtered_data = {algoid:details  for algoid, details in ourdata.items()  if 'sociale zekerheid' in details.get('category').lower()}
# ...but let's just work on all instead  (right now we still filter later, based on whether we found any interesting words)
filtered_data = ourdata

In [20]:
def find_whole(pattern, text): # helper function
    ' let us match by substring that ends in a word, but return the rest of the word as well,   e.g. add_whole( "soci", "sociale zekerheid" ) == "sociale" '
    match = re.search(r'\b'+pattern+r'[\w]*', text)
    if match is not None:
        return [match.group(0)]
    else:
        return []

for algo_id, details in filtered_data.items():
    category = details.get('category')
    textlist = details.get('textlist')
    text = '  '.join(textlist).lower()  

    # find interesting words, add to the case's fields
    for szi in social_indicators:
        details['sociaal'].update(    find_whole( szi, text )  )
    for wi  in worrisome_indicators:
        details['worrying'].update(   find_whole( wi,  text )  )
    for ai  in alleviating_indicators:
        details['alleviates'].update( find_whole( ai,  text )  )

    # WARNING: DUMB SCORE -- to calculate anything even resembling a real store would take a lot more than this
    score = round(   math.log( 1+ len(details['sociaal']))  +  math.log(1+len(details['worrying']))  -  len(details['alleviates'] ),   1)
    details['dumb_review_score'] = round(score,1)

## Show in a nice table

We use pandas again, mostly because it lets us control formatting

In [21]:
with pandas.option_context('display.min_rows', 1000, 'display.max_rows', 1000): # don't truncate on viewing this table
    df = pandas.DataFrame(filtered_data.values()) # make it a pandas table again

    display( df
        [['category', 'algo_id', 'titel', 'sociaal', 'worrying', 'alleviates', 'dumb_review_score']] # filters out 'textlist' and also forces column ordering  order
        [ (df['sociaal'].str.len()>0) ] # select only items that had any words indicating it's a social security thing:   NOTE: comment out this line to see ALL algorithms
        .sort_values(['category','dumb_review_score'],ascending=False) # sort by category, then score (to see the 'Sociale zekerheid' cases grouped / see most troublesome per group)
        
        # display the table slightly prettier:
        .style.format({ 
            'algo_id':    lambda x:'<a href="https://algoritmes.overheid.nl/nl/algoritme/IGNOREME/%s#algemeneInformatie">%s</a>'%(x,x), # make algorithm IDs link to the website
            'sociaal':    lambda x:', '.join(x),    # (the rest of these are minor formatting, that we could do without)
            'worrying':   lambda x:', '.join(x),
            'alleviates': lambda x:', '.join(x),
            'dumb_review_score': lambda x:'%.1f'%x
        })
        .hide(axis="index") # hide pandas's index column
    )

category,algo_id,titel,sociaal,worrying,alleviates,dumb_review_score
Zorg en gezondheid,13669212,Wmo voorspelmodel,"de wmo, hulp bij het huishouden, ondersteuning thuis","huishouden, risico, voorspelmodel, combineren, leeftijdscategorieën, individuele","privacy, ondersteuning",1.3
Zorg en gezondheid,31934363,Signalering Misbruik en Oneigenlijk gebruik Wmo & Jeugdwet,"zorgaanbieder, de wmo","risico, onderzoek, profilering, bsn, brp, beslissing","profilering, handmatige",1.0
Werk,42117256,Risicomodel omzetdaling NOW,uwv,"onderzoek, voorspellingen, functies, individueel, dossiers, risicomodel, beslissing","ondersteunen, adviseert",0.8
"Sociale zekerheid, Werk, Overheidsfinanciën",86726997,Toekenning Bijverdienbeloning,"bijstandsuitkering, sociale zekerheid, ioaz, uitkering, ioaw, participatiewet","financiele gegevens, risico, bijstandsuitkering, dossiersniveau, toeslagen",,3.7
"Sociale zekerheid, Werk",15759870,Risicoscan Verwijtbare Werkloosheid,"sociale zekerheid, uitkering, uwv","risicoscan, leeftijd, onderzoek, dossier",privacy,2.0
"Sociale zekerheid, Werk",11248112,Werkverkenner,"sociale zekerheid, arbeidsongeschikt, uitkering, uwv","functie, beslist, leeftijd, score, werkzoekenden, bsn, uw gegevens, burgerservicenummer","ondersteunen, privacy, advies",0.8
"Sociale zekerheid, Werk",79826945,Sollicitatiescan WW,"sociale zekerheid, uitkering, uwv","onderzoeken, beslist, leeftijd, score, bsn, burgerservicenummer, uw gegevens, persoonsgegevens","privacy, ondersteuning, adviesbureau",0.6
"Sociale zekerheid, Organisatie en bedrijfsvoering",65732191,Sociaal Domein: PKO Kennissystemen (Proces & Kennisondersteuning),"uitkeringsaanvragen, sociale zekerheid, bijstand, werk en inkomen","bijstand, risico, beslissen, individuele, brp, heronderzoeken, beslisboom",ondersteunt,2.7
Sociale zekerheid,19118228,Sociaal Domein: eDiensten voor aanvragen,"sociale zekerheid, bijstand, uitkeringen","bijstand, risico, gestuurd, bsn, brp, burgerservicenummer, beslisbomen",,3.5
Sociale zekerheid,92734630,Uitkering Participatiewet,"sociale zekerheid, werk en inkomen, uitkering, participatiewet","zonder tussenkomst van, risico, persoonsgegevens, beslissing",,3.2
